In [1]:
import heapq

class Node:
    def __init__(self, position, g, h):
        self.position = position  # (x, y) position
        self.g = g  # cost from start to current node
        self.h = h  # heuristic cost to goal
        self.f = g + h  # total cost (f = g + h)

    def __lt__(self, other):
        return self.f < other.f  # for priority queue

def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_algorithm(grid):
    start = (0, 0)
    goal = (4, 4)
    open_list = []
    closed_list = set()

    start_node = Node(start, 0, manhattan_distance(start, goal))
    heapq.heappush(open_list, start_node)

    # Directions: right, down, left, up
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    while open_list:
        current_node = heapq.heappop(open_list)

        if current_node.position == goal:
            return current_node.g  # Return cost if goal is reached

        closed_list.add(current_node.position)

        for direction in directions:
            neighbor_pos = (current_node.position[0] + direction[0], current_node.position[1] + direction[1])

            if (0 <= neighbor_pos[0] < 5) and (0 <= neighbor_pos[1] < 5) and grid[neighbor_pos[0]][neighbor_pos[1]] == 0:
                if neighbor_pos in closed_list:
                    continue

                g_cost = current_node.g + 1
                h_cost = manhattan_distance(neighbor_pos, goal)
                neighbor_node = Node(neighbor_pos, g_cost, h_cost)

                if neighbor_node not in open_list or neighbor_node.g < current_node.g:
                    heapq.heappush(open_list, neighbor_node)

    return None  # Return None if no path is found

def reconstruct_path(grid, path):
    for p in path:
        grid[p[0]][p[1]] = 'P'  # Mark path with 'P'

# Example grid with obstacles
grid = [
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0]
]

path_cost = a_star_algorithm(grid)
print("Path Cost:", path_cost)

# Marking path on the grid
reconstruct_path(grid, [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (4, 4)])  # Example path
for row in grid:
    print(row)


Path Cost: 8
['P', 'P', 'P', 'P', 0]
[0, 1, 1, 'P', 0]
[0, 0, 0, 'P', 0]
[0, 1, 0, 'P', 0]
[0, 0, 0, 'P', 'P']


In [6]:
import heapq

class WaterJugState:
    """Class representing the state of the water jugs."""

    def __init__(self, jug1, jug2, steps=0):
        self.jug1 = jug1  # Current amount in jug 1
        self.jug2 = jug2  # Current amount in jug 2
        self.steps = steps  # Steps taken to reach this state

    def __hash__(self):
        return hash((self.jug1, self.jug2))  # Hash function for storing in a set

    def __eq__(self, other):
        return self.jug1 == other.jug1 and self.jug2 == other.jug2  # Equality comparison

    def __lt__(self, other):
        """Define less than for priority comparison based on steps."""
        return (self.jug1, self.jug2) < (other.jug1, other.jug2)

def water_jug_a_star(capacity1, capacity2, target):
    """A* algorithm to solve the water jug problem.

    Args:
        capacity1 (int): Capacity of jug 1.
        capacity2 (int): Capacity of jug 2.
        target (int): Target amount to measure.

    Returns:
        int: Minimum steps to achieve the target amount, or None if not possible.
    """
    # Initialize the initial state with both jugs empty
    initial_state = WaterJugState(0, 0)
    queue = []  # Priority queue for A* search
    visited = set()  # Set to keep track of visited states

    # Push the initial state into the priority queue with 0 cost
    heapq.heappush(queue, (0, initial_state))

    while queue:
        current_cost, current_state = heapq.heappop(queue)  # Get the state with the lowest cost

        # Check if we have reached the target
        if current_state.jug1 == target or current_state.jug2 == target:
            return current_state.steps  # Return the number of steps if target is reached

        # Skip processing if the current state has already been visited
        if current_state in visited:
            continue
        visited.add(current_state)  # Mark this state as visited

        # List of possible actions leading to new states
        actions = [
            WaterJugState(capacity1, current_state.jug2, current_state.steps + 1),  # Fill jug 1
            WaterJugState(current_state.jug1, capacity2, current_state.steps + 1),  # Fill jug 2
            WaterJugState(0, current_state.jug2, current_state.steps + 1),  # Empty jug 1
            WaterJugState(current_state.jug1, 0, current_state.steps + 1),  # Empty jug 2
            WaterJugState(max(current_state.jug1 - (capacity2 - current_state.jug2), 0),
                          min(capacity2, current_state.jug1 + current_state.jug2),
                          current_state.steps + 1),  # Pour jug 1 into jug 2
            WaterJugState(min(capacity1, current_state.jug1 + current_state.jug2),
                          max(current_state.jug2 - (capacity1 - current_state.jug1), 0),
                          current_state.steps + 1)  # Pour jug 2 into jug 1
        ]

        for action in actions:
            if action not in visited:  # Only consider unvisited states
                # Heuristic: absolute difference from target amount
                heuristic = abs(action.jug1 - target) + abs(action.jug2 - target)
                heapq.heappush(queue, (action.steps + heuristic, action))  # Push new state into the queue

    return None  # Return None if no solution is found

# Set the target amount and jug capacities
target_amount = 2
result = water_jug_a_star(4, 3, target_amount)  # Jug capacities of 4 liters and 3 liters

# Output the result
if result is not None:
    print("Steps to achieve target amount:", result)
else:
    print("It is not possible to measure the target amount.")


Steps to achieve target amount: 4


In [7]:
import random

class EightQueens:
    def __init__(self, size=8):
        self.size = size  # Size of the chessboard (8x8)
        self.queens = self.initial_state()  # Initial state with queens

    def initial_state(self):
        """Generate an initial random state with one queen per column."""
        # Randomly place one queen in each column
        return [random.randint(0, self.size - 1) for _ in range(self.size)]

    def heuristic(self, state):
        """Calculate the number of pairs of queens that are attacking each other."""
        conflicts = 0
        for i in range(self.size):
            for j in range(i + 1, self.size):
                # Check if queens in columns i and j are attacking each other
                if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                    conflicts += 1
        return conflicts  # Return the number of conflicts

    def get_neighbors(self, state):
        """Generate all possible neighboring states."""
        neighbors = []
        for col in range(self.size):
            for row in range(self.size):
                if row != state[col]:  # Change the row of the queen in column 'col'
                    neighbor = state[:]
                    neighbor[col] = row  # Move the queen to the new row
                    neighbors.append(neighbor)
        return neighbors

    def solve(self):
        """Solve the 8-Queens problem using the Hill-Climbing algorithm."""
        current_state = self.queens
        current_conflicts = self.heuristic(current_state)

        while current_conflicts > 0:  # Until there are no conflicts
            neighbors = self.get_neighbors(current_state)  # Get all neighbors
            next_state = None
            next_conflicts = float('inf')  # Initialize next conflicts to a high number

            for neighbor in neighbors:
                # Evaluate each neighbor's heuristic
                conflict_count = self.heuristic(neighbor)
                # Select the neighbor with the least number of conflicts
                if conflict_count < next_conflicts:
                    next_conflicts = conflict_count
                    next_state = neighbor

            # If no better neighbor is found, exit the loop
            if next_conflicts >= current_conflicts:
                break

            current_state = next_state  # Move to the better neighbor
            current_conflicts = next_conflicts  # Update current conflicts

        return current_state if current_conflicts == 0 else None  # Return solution or None if unsolvable

# Create an instance of the EightQueens class and solve the problem
eight_queens = EightQueens()
solution = eight_queens.solve()

# Output the solution
if solution:
    print("Solution found:")
    for row in solution:
        print(" " * row + "Q")  # Print the board with queens
else:
    print("No solution found.")


No solution found.


In [10]:
import numpy as np

# Constants for the game
PLAYER_X = "X"  # Human player
PLAYER_O = "O"  # AI player

class TicTacToe:
    def __init__(self):
        """Initialize the Tic-Tac-Toe board."""
        self.board = np.full((3, 3), " ")  # Create a 3x3 board filled with empty spaces
        self.current_player = PLAYER_X  # Set the current player to X

    def print_board(self):
        """Print the current board state."""
        print("\n".join([" | ".join(row) for row in self.board]))
        print()

    def check_winner(self):
        """Check for a winner in the game."""
        # Check rows, columns, and diagonals for a winning condition
        for player in [PLAYER_X, PLAYER_O]:
            if any(np.all(self.board[i] == player) for i in range(3)) or \
               any(np.all(self.board[:, i] == player) for i in range(3)) or \
               (np.all(np.diag(self.board) == player) or np.all(np.diag(np.fliplr(self.board)) == player)):
                return player
        if np.all(self.board != " "):
            return "Draw"  # Return 'Draw' if the board is full
        return None  # No winner yet

    def minimax(self, depth, is_maximizing):
        """The Mini-Max algorithm for AI to determine the best move."""
        winner = self.check_winner()
        if winner == PLAYER_O:
            return 1  # AI wins
        elif winner == PLAYER_X:
            return -1  # Player X wins
        elif winner == "Draw":
            return 0  # Draw

        if is_maximizing:
            best_score = -np.inf
            for i in range(3):
                for j in range(3):
                    if self.board[i][j] == " ":
                        self.board[i][j] = PLAYER_O  # AI move
                        score = self.minimax(depth + 1, False)  # Recursively call minimax
                        self.board[i][j] = " "  # Undo the move
                        best_score = max(best_score, score)  # Update best score
            return best_score
        else:
            best_score = np.inf
            for i in range(3):
                for j in range(3):
                    if self.board[i][j] == " ":
                        self.board[i][j] = PLAYER_X  # Player X move
                        score = self.minimax(depth + 1, True)  # Recursively call minimax
                        self.board[i][j] = " "  # Undo the move
                        best_score = min(best_score, score)  # Update best score
            return best_score

    def best_move(self):
        """Determine the best move for the AI."""
        best_score = -np.inf
        move = (-1, -1)  # Default move
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == " ":
                    self.board[i][j] = PLAYER_O  # AI move
                    score = self.minimax(0, False)  # Get score using minimax
                    self.board[i][j] = " "  # Undo the move
                    if score > best_score:  # If this move is better than the best score
                        best_score = score
                        move = (i, j)  # Update best move
        return move

    def play(self):
        """Start the Tic-Tac-Toe game."""
        while True:
            self.print_board()  # Print current board state
            if self.current_player == PLAYER_X:
                row, col = map(int, input("Enter your move (row and column): ").split())
                if self.board[row][col] == " ":
                    self.board[row][col] = PLAYER_X  # Player X's move
                    if self.check_winner():
                        break  # Exit if there's a winner
                    self.current_player = PLAYER_O  # Switch to AI player
                else:
                    print("Invalid move! Try again.")
            else:
                print("AI is making a move...")
                row, col = self.best_move()  # Get best move from AI
                self.board[row][col] = PLAYER_O  # AI's move
                if self.check_winner():
                    break  # Exit if there's a winner
                self.current_player = PLAYER_X  # Switch to human player

        # Print the final board and result
        self.print_board()
        winner = self.check_winner()
        if winner == "Draw":
            print("The game is a draw!")
        else:
            print(f"The winner is {winner}!")

# Start the game
if __name__ == "__main__":
    game = TicTacToe()
    game.play()


  |   |  
  |   |  
  |   |  

Enter your move (row and column): 2 1
  |   |  
  |   |  
  | X |  

AI is making a move...
  | O |  
  |   |  
  | X |  

Enter your move (row and column): 1 2
  | O |  
  |   | X
  | X |  

AI is making a move...
  | O |  
  |   | X
O | X |  

Enter your move (row and column): 0 0
X | O |  
  |   | X
O | X |  

AI is making a move...
X | O |  
  | O | X
O | X |  

Enter your move (row and column): 2 0
Invalid move! Try again.
X | O |  
  | O | X
O | X |  

Enter your move (row and column): 1 2
Invalid move! Try again.
X | O |  
  | O | X
O | X |  

Enter your move (row and column): 0 2
X | O | X
  | O | X
O | X |  

AI is making a move...
X | O | X
  | O | X
O | X | O

Enter your move (row and column): 2 2
Invalid move! Try again.
X | O | X
  | O | X
O | X | O

Enter your move (row and column): 1 0
X | O | X
X | O | X
O | X | O

The game is a draw!
